<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1


    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63


    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-27 10:26:50
-------------------
qualified stocks: 93
with latest results: 30
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.35 C
-------------------
Today PnL: 81.85 K (0.61%)
Current PnL: -14.49 L (-10.1%)
CY Booked + Current PnL: -8.00 L (-5.57%)
-------------------
Total profit:  3.98 L
Total loss:  -18.48 L
-------------------
Total Booked + Current PnL: 18.98 L (16.28%)
Total Booked PnL: 33.47 L (28.72%)
Curr Year Booked PnL: 6.49 L (4.81%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 71.37 L (52.94%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 9.99%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
62    RELIANCE  1291.83  1526.00    10.27       X-LC  68.82  163544.0   
3   ADANIPORTS  1103.69  1583.00     2.87       M-LC  54.84  103021.0   
84  UJJIVANSFB    52.77    53.00    52.86       M-SC  84.95  132300.0   
35    HINDALCO   651.95   761.55    -7.59       H-LC  64.52  111040.0   
78   TATAELXSI  7332.28  7332.00   -14.97       X-MC  36.56   89250.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
62      22735.0   2797.0        0.34         16.15   1.71  18.13     18.0   
3       24659.0   9375.0        1.52         31.47   9.10  43.43     79.0   
84     -10179.0  10796.0       -0.20         -7.14   8.16   0.44    242.0   
35       6728.0  10804.0        0.49          6.45   9.73  16.81     27.0   
78     -13402.0  13396.0        1.60        -13.06  15.01  -0.00     58.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
62     8.13        1.21    29.13     XY25      NTT  REFINERIES  
3      2.63        0.76    38.46     XY24      NTT        MISC  
84    -0.94        0.98    56.15    OX40N      NTT       BANKS  
35     0.62        0.82    23.53      X5K      ATH      METALS  
78    -1.00        0.66    35.67    OX40N      NTT          IT

In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
50    KPIGREEN   497.21   732.16      NaN        NaN  72.04  125471.0   
34  HEROMOTOCO  4311.81  6039.03    -0.95       H-MC  32.26  151263.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
50        174.0  59034.0        0.15          0.14  47.05  47.25    137.0   
34        350.0  60097.0        1.02          0.23  39.73  40.06    102.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
50     0.00        0.93    56.97       MH      ATH    POWER  
34     0.01        1.12    23.72       AR      ATH     AUTO

In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol   AvgCost       FTT  Dev%_PE Conviction    RSP   Current  \
75   SURYODAY    189.78    240.00    57.89       H-SC  77.42  112441.0   
71    SIEMENS   4406.90   7969.85   -10.35       X-LC  24.73   81178.0   
26      DIXON  14367.00  18940.15   -11.12       H-LC  21.51  117424.0   
19       CERA   8421.60   8422.00   -14.80       X-SC  61.29   87886.0   
12  BANKINDIA    116.91    190.00   -30.66       M-MC  70.97  183637.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
75     -37485.0   77157.0        3.67        -25.00   68.62  26.46    143.0   
71     -28994.0  118073.0        2.60        -26.32  145.45  80.85     12.0   
26       2488.0   34100.0        2.54          2.16   29.04  31.83     99.0   
19     -21595.0   21602.0        2.49        -19.72   24.58   0.00     68.0   
12       3829.0  108585.0        2.09          2.13   59.13  62.52    170.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
75    -0.49        0.83    44.04       XR      NTT       BANKS  
71    -0.25        0.60    19.56       AR      ATH  ELECTRICAL  
26     0.07        0.87    34.58     X40N      ATH          IT  
19    -1.00        0.65    33.02    OX40N      NTT    CERAMICS  
12     0.04        1.36    36.94       XR      NTT       BANKS

In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT   Dev%_PE Conviction     RSP   Current  \
4   ALKYLAMINE  2347.98  4546.37      3.55       X-SC   97.85   96346.0   
15   BRITANNIA  4983.33  6446.05     15.59       X-LC   60.22  264937.0   
46       JCHAC  2282.24  2282.00  20117.39       M-SC   52.69   98840.0   
7   ASIANTILES    75.41   137.00   7183.33       M-SC  100.00   79939.0   
60  RAJESHEXPO   517.65   518.00   1858.33       M-SC   46.24   53437.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
4       -4617.0  99150.0       -1.31         -4.57  102.91  93.63     65.0   
15      35704.0  31580.0       -1.18         15.58   11.92  29.35      7.0   
46     -28965.0  28950.0       -0.95        -22.66   29.29  -0.01    220.0   
7      -13871.0  90491.0       -0.85        -14.79  113.20  81.67    240.0   
60     -83740.0  83832.0       -0.82        -61.05  156.88   0.07    232.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
4     -0.05        0.71    43.50       SR      ATH  CHEMICALS  
15     1.13        1.97    25.89     XY25      ATH       FMCG  
46    -1.00        0.73     9.70    OX40N      NTT         AC  
7     -0.15        0.59    54.81       XR      NTT   CERAMICS  
60    -1.00        0.40    32.90    OX40N      NTT  JEWELLERY

In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
84  UJJIVANSFB    52.77    53.0    52.86       M-SC  84.95  132300.0   
78   TATAELXSI  7332.28  7332.0   -14.97       X-MC  36.56   89250.0   
89      VIPIND   488.80   489.0  -954.99       H-SC  96.77   81461.0   
42  INDIGOPNTS  1407.73  1408.0   120.98       H-SC  89.25  145068.0   
76    SYMPHONY  1306.42  1306.0   -33.36       M-SC   1.08  142004.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%  OTT%  CumlRnk  \
84     -10179.0  10796.0       -0.20         -7.14   8.16  0.44    242.0   
78     -13402.0  13396.0        1.60        -13.06  15.01 -0.00     58.0   
89     -13855.0  13897.0        0.25        -14.54  17.06  0.04    153.0   
42     -29491.0  29521.0        0.34        -16.89  20.35  0.02    128.0   
76     -29137.0  29082.0        0.33        -17.03  20.48 -0.03    188.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
84    -0.94        0.98    56.15    OX40N      NTT     BANKS  
78    -1.00        0.66    35.67    OX40N      NTT        IT  
89    -1.00        0.60    60.67    OX40N      NTT      MISC  
42    -1.00        1.08    25.80    OX40N      NTT    PAINTS  
76    -1.00        1.05     2.06    OX40N      NTT  DURABLES

In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
80         TCS  3794.03  4998.00   -17.44       X-LC  31.18  269256.0   
44        INFY  1461.46  2275.00    -9.55       X-LC  59.14  290177.0   
36  HINDUNILVR  2413.81  2723.00   -18.24       X-LC  22.58  256973.0   
15   BRITANNIA  4983.33  6446.05    15.59       X-LC  60.22  264937.0   
57   NESTLEIND  2268.60  2755.00    -9.80       X-LC  30.11  286463.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
80     -26678.0  120600.0        0.30         -9.01  44.79  31.73      1.0   
44      28576.0  117057.0        0.32         10.92  40.34  55.67      2.0   
36     -13374.0   48003.0        0.60         -4.95  18.68  12.81      6.0   
15      35704.0   31580.0       -1.18         15.58  11.92  29.35      7.0   
57      21037.0   35865.0        0.70          7.93  12.52  21.44      9.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
80    -0.22        2.00     7.77      X40      BTT       IT  
44     0.24        2.15    17.70      X40      BTT       IT  
36    -0.28        1.91     7.47     XY25      NTT     FMCG  
15     1.13        1.97    25.89     XY25      ATH     FMCG  
57     0.59        2.13    14.61     XY25      NTT     FMCG

In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
91   WHIRLPOOL  1167.49  2270.0   -28.36       M-SC  90.32  208902.0   
85   UNIONBANK   123.87   163.0    -1.84       M-LC  74.19  171448.0   
3   ADANIPORTS  1103.69  1583.0     2.87       M-LC  54.84  103021.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
91      37281.0  124798.0        0.96         21.72  59.74  94.43    196.0   
85      30608.0   13887.0        1.80         21.73   8.10  31.59     90.0   
3       24659.0    9375.0        1.52         31.47   9.10  43.43     79.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
91     0.30        1.55    54.18       XR      NTT  DURABLES  
85     2.20        1.27    48.93     XY24      NTT     BANKS  
3      2.63        0.76    38.46     XY24      NTT      MISC

In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
65        SAIL   130.64   228.00   103.46       M-MC  58.06  231643.0   
18        CAMS  3643.00  5250.99     6.89       H-SC  80.65  120238.0   
3   ADANIPORTS  1103.69  1583.00     2.87       M-LC  54.84  103021.0   
47      JIOFIN   310.58   387.00    -0.39       M-LC  78.49  220010.0   
34  HEROMOTOCO  4311.81  6039.03    -0.95       H-MC  32.26  151263.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
65       6681.0  160969.0        0.67          2.97  69.49  74.53    186.0   
18      18234.0   26789.0        1.93         17.88  22.28  44.14    124.0   
3       24659.0    9375.0        1.52         31.47   9.10  43.43     79.0   
47       2604.0   50888.0        0.58          1.20  23.13  24.61     94.0   
34        350.0   60097.0        1.02          0.23  39.73  40.06    102.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
65     0.04        1.72    34.49     XY24      BTT    STEEL  
18     0.68        0.89    37.02     X40N      ATH     MISC  
3      2.63        0.76    38.46     XY24      NTT     MISC  
47     0.05        1.63    56.43     XY24      BTT  FINANCE  
34     0.01        1.12    23.72       AR      ATH     AUTO

In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
91   WHIRLPOOL  1167.49  2270.00   -28.36       M-SC  90.32  208902.0   
92       WIPRO   243.46   420.00    -2.90       M-LC  81.72  166303.0   
41   INDIAMART  2327.09  4850.92   -46.05       H-SC  94.62  140450.0   
56  NATIONALUM   189.63   247.44   -34.88       H-SC  76.34  103345.0   
12   BANKINDIA   116.91   190.00   -30.66       M-MC  70.97  183637.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  CumlRnk  \
91      37281.0  124798.0        0.96         21.72  59.74   94.43    196.0   
92      15358.0   94094.0       -0.11         10.17  56.58   72.51     69.0   
41      17114.0  116644.0        0.48         13.88  83.05  108.45    121.0   
56       2841.0   27800.0        0.72          2.83  26.90   30.49     97.0   
12       3829.0  108585.0        2.09          2.13  59.13   62.52    170.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
91     0.30        1.55    54.18       XR      NTT  DURABLES  
92     0.16        1.23    14.55       XR      NTT        IT  
41     0.15        1.04    39.33       AR      ATH      MISC  
56     0.10        0.77    36.15       MH      ATH    METALS  
12     0.04        1.36    36.94       XR      NTT     BANKS

In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
22   COFFEEDAY    59.14    80.00   -51.71       L-SC  93.55   72710.0   
54      MASFIN   326.60   402.39   -18.70       H-SC  67.74   92490.0   
4   ALKYLAMINE  2347.98  4546.37     3.55       X-SC  97.85   96346.0   
3   ADANIPORTS  1103.69  1583.00     2.87       M-LC  54.84  103021.0   
87  VALIANTORG   512.64   838.00  -358.27       H-SC  43.01  109798.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
22     -40839.0   80890.0        0.53        -35.97  111.25  35.27    261.0   
54      -5490.0   28228.0       -0.21         -5.60   30.52  23.21    141.0   
4       -4617.0   99150.0       -1.31         -4.57  102.91  93.63     65.0   
3       24659.0    9375.0        1.52         31.47    9.10  43.43     79.0   
87     -23488.0  108085.0       -0.49        -17.62   98.44  63.47    145.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
22    -0.50        0.54    77.96       XR      NTT     HOTELS  
54    -0.19        0.69    34.14       XR      ATH    FINANCE  
4     -0.05        0.71    43.50       SR      ATH  CHEMICALS  
3      2.63        0.76    38.46     XY24      NTT       MISC  
87    -0.22        0.81    84.05       XR      NTT  CHEMICALS

In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction     RSP   Current  \
7   ASIANTILES    75.41   137.00  7183.33       M-SC  100.00   79939.0   
48     JPPOWER    18.73    26.20   -32.62       L-SC   98.92  136541.0   
4   ALKYLAMINE  2347.98  4546.37     3.55       X-SC   97.85   96346.0   
89      VIPIND   488.80   489.00  -954.99       H-SC   96.77   81461.0   
66  SAMMAANCAP   170.35   326.00  -170.94       M-SC   95.70   85368.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
7      -13871.0   90491.0       -0.85        -14.79  113.20  81.67    240.0   
48      -5301.0   61867.0        0.33         -3.74   45.31  39.88    259.0   
4       -4617.0   99150.0       -1.31         -4.57  102.91  93.63     65.0   
89     -13855.0   13897.0        0.25        -14.54   17.06   0.04    153.0   
66     -16842.0  110236.0       -0.76        -16.48  129.13  91.37    203.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
7     -0.15        0.59    54.81       XR      NTT   CERAMICS  
48    -0.09        1.01    38.37     XY24      NTT      POWER  
4     -0.05        0.71    43.50       SR      ATH  CHEMICALS  
89    -1.00        0.60    60.67    OX40N      NTT       MISC  
66    -0.15        0.63    38.58     XY25      NTT    FINANCE

In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.21
1     25       42.47
2     50       71.10

In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.59
LC    37.20
MC    21.57
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.28
XY25     13.38
X40      12.39
X40N     10.59
XR       10.36
OX40N     8.25
AR        7.89
MH        1.70
X5K       1.49
X200      1.43
SR        1.20
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.42
X-LC    15.38
M-SC    13.96
H-LC    12.41
M-LC     8.32
H-MC     8.12
X-MC     7.43
M-MC     5.66
X-SC     2.74
L-SC     2.47
L-LC     1.09
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                15.32         -1.42   35.36
IT                  11.95        -10.08   64.73
BANKS                7.97         -6.02   49.28
FINANCE              7.95        -12.65   53.42
MISC                 6.66        -20.18   67.05
PAINTS               5.63        -15.33   38.07
ELECTRICAL           5.08         -2.69   47.16
HEALTHCARE           4.50         -6.26   37.29
AUTO                 4.41        -13.76   63.78
INSURANCE            3.32         -5.64   39.18
DURABLES             3.21         -2.48   39.94
FOOTWEAR             2.34        -30.93   74.20
METALS               2.26         -0.92   30.84
AC                   2.21         -7.35   39.60
POWER                1.94         -1.96   46.14
CERAMICS             1.76        -27.51   70.46
STEEL                1.72          2.88   69.49
TRAVEL               1.68        -40.88  101.84
CHEMICALS            1.52        -13.63  100.53
ENGINEERING          1.24        -14.73   61.87
REFINERIES           1.21         13.90    1.71
JEWELLERY            1.17        -83.79  130.96
MINING               1.09         -2.68   29.31
CEMENT               1.04        -34.26  103.34
ENTERTAINMENT        0.92        -30.55   96.19
PHARMA               0.74          0.21   19.81
TEXTILES             0.58        -18.38   52.14
HOTELS               0.54        -56.17  111.25

In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      2717010.0     25
XR        1060573.0     13
AR        1004150.0      9
X40        652311.0      9
X40N       501457.0     10
XY25       482824.0      9
OX40N      317181.0     11
SR         181539.0      2
MH          86834.0      2
X200        67265.0      1
X5K         66254.0      2

In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        1878804.0     19
M-SC        1445399.0     18
X-LC         592952.0      9
M-MC         581879.0      5
H-LC         580972.0     11
H-MC         427544.0      6
X-MC         413462.0      6
M-LC         320161.0      6
L-SC         263173.0      4
X-SC         261958.0      4
L-MC          54715.0      1
L-LC          42950.0      1

In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      861494.0      6
M-SC       XY24      596351.0      5
H-SC       AR        516052.0      3
M-MC       XY24      381469.0      3
M-SC       XR        325001.0      4
X-LC       X40       313120.0      3
H-LC       X40N      232178.0      5
H-MC       XY24      226297.0      2
H-SC       XR        213470.0      3
M-LC       XY24      210925.0      4
M-SC       OX40N     169989.0      5
H-SC       X40N      166782.0      3
X-MC       XY24      164212.0      2
H-LC       X40       145214.0      2
L-SC       XR        142490.0      2
X-SC       XR        122218.0      1
X-LC       XY25      118245.0      4
           AR        118073.0      1
M-SC       XY25      110236.0      1
M-MC       XR        108585.0      1
X-MC       XY25      107310.0      1
X-SC       SR         99150.0      1
M-LC       XR         94094.0      1
H-SC       OX40N      93206.0      3
M-MC       AR         91825.0      1
M-SC       AR         89226.0      1
H-MC       XY25       88941.0      1
M-SC       SR         82389.0      1
           X40        72207.0      1
H-LC       AR         70061.0      1
X-MC       X40        69561.0      1
H-LC       X200       67265.0      1
           X5K        66254.0      2
L-SC       XY24       61867.0      1
H-MC       AR         60097.0      1
X-MC       X40N       58983.0      1
L-SC       AR         58816.0      1
L-MC       XR         54715.0      1
H-MC       X40        52209.0      2
X-LC       X40N       43514.0      1
L-LC       XY25       42950.0      1
X-SC       OX40N      40590.0      2
H-SC       MH         27800.0      1
M-LC       XY25       15142.0      1
X-MC       OX40N      13396.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
